In [11]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph,Kingdom,Occurrence
from drivers.graph_models import Cell,Mex4km, countObjectsOf
from drivers.graph_models import pickNode
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import numpy as np

## Use the ggplot style
plt.style.use('ggplot')

In [2]:
## Pick a node, e.g. Pines
## Let's pick the bats node
pines = pickNode(Order,name='Pinales')

In [3]:
%time idspines = pines.getCellsById()

CPU times: user 104 ms, sys: 8 ms, total: 112 ms
Wall time: 111 ms


In [4]:
selected_cells = idspines

In [5]:
from traversals import sampling as sm
cells_with_pines = pines.is_in.related_class
N = 300
sample_cells = sm.UniformRandomCellSample(idspines,cells_with_pines,sample_size=N)

INFO Compiling Query and asking the Graph Database


In [6]:
from traversals import strategies as st

In [7]:
%time data = st.getEnvironmentalCovariatesFromListOfCells(sample_cells)

CPU times: user 21.2 s, sys: 380 ms, total: 21.6 s
Wall time: 2min 21s


In [9]:
data

,Elevation_mean,MaxTemperature_mean,MeanTemperature_mean,MinTemperature_mean,Precipitation_mean,SolarRadiation_mean,Vapor_mean,WindSpeed_mean,Longitude,Latitude
0,2312.000000,24.346528,15.618056,24.402778,37.872685,16873.078704,1.000000,3.988426,-101.571563,24.65239
1,1175.555556,27.612222,20.516667,27.652778,62.902778,17450.591667,1.575000,2.116667,-99.219563,23.32939
2,1859.111111,25.722454,17.726852,25.766204,31.266204,18480.472222,1.143519,2.416667,-101.032563,21.76139
3,907.555556,27.068056,20.500000,27.044444,25.688889,16589.738889,1.416667,3.000000,-101.326563,26.61239
4,1866.000000,24.846759,19.328704,24.879630,87.136574,17439.145833,1.416667,2.576389,-96.867563,16.22439
5,2438.222222,21.156482,11.273148,21.266204,68.453704,17752.553241,0.921296,3.152778,-105.932563,25.19139
6,328.444444,30.389352,25.500000,30.402778,105.828704,17486.310185,2.414352,3.331019,-94.711563,16.66539
7,3135.666667,16.631250,10.462963,16.664352,81.275463,17374.877315,1.000000,2.886574,-97.847563,19.50739
8,2079.222222,23.781944,15.141204,23.844907,31.662037,17093.682870,1.000000,3.949074,-100.346563,24.65239
9,1831.666667,25.850463,20.238426,25.907407,104.062500,17975.083333,1.340278,1.960648,-101.375563,18.23339


## Recreate Local Trees 

In [13]:
from drivers.tree_builder import buildTreeNeo

In [15]:
%time trees = map(lambda c : buildTreeNeo(c),sample_cells)

CPU times: user 7min, sys: 11.6 s, total: 7min 11s
Wall time: 7min 45s


## Calculate big Tree


In [17]:
%time ocs = reduce(lambda a,b : a + b ,map(lambda t : t.occurrences, trees))

CPU times: user 456 ms, sys: 0 ns, total: 456 ms
Wall time: 454 ms


In [19]:
## Ohh! super fast (compared to the other method that takes more than 20 minutes for this sample size)
%time bigtree = TreeNeo(ocs,cell_objects=selected_cells)

CPU times: user 3.45 s, sys: 12 ms, total: 3.46 s
Wall time: 3.47 s


In [20]:
bigtree

<LocalTree Of Life | Root: LUCA - n.count : 80065- >

In [21]:
bigtree.countNodesFrequenciesOnList(trees)

INFO Going deep 0.00666666666667
INFO Going deep 0.00666666666667
INFO Going deep 0.00666666666667
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00666666666667
INFO Going deep 0.00333333333333
INFO Going deep 0.01
INFO Going deep 0.00666666666667
INFO Going deep 0.01
INFO Going deep 0.00333333333333
INFO Going deep 0.00666666666667
INFO Going deep 0.02
INFO Going deep 0.02
INFO Going deep 0.00333333333333
INFO Going deep 0.01
INFO Going deep 0.01
INFO Going deep 0.01
INFO Going deep 0.0233333333333
INFO Going deep 0.0233333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.0333333333333
INFO Going deep 0.0333333333333

0.05

## Animals associated with Pines

In [22]:
animals = bigtree.to_Animalia.plantTreeNode()

In [40]:
animals.countNodesFrequenciesOnList(list_of_trees=trees)

INFO Going deep 0.00666666666667
INFO Going deep 0.00666666666667
INFO Going deep 0.00666666666667
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00666666666667
INFO Going deep 0.00333333333333
INFO Going deep 0.01
INFO Going deep 0.00666666666667
INFO Going deep 0.01
INFO Going deep 0.00333333333333
INFO Going deep 0.00666666666667
INFO Going deep 0.02
INFO Going deep 0.02
INFO Going deep 0.00333333333333
INFO Going deep 0.01
INFO Going deep 0.01
INFO Going deep 0.01
INFO Going deep 0.0233333333333
INFO Going deep 0.0233333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.00333333333333
INFO Going deep 0.0333333333333
INFO Going deep 0.0333333333333

0.05

In [41]:
animals.families.sort(key=lambda k:k.n_presences_in_list,reverse=True)

### Pseudo presence absences

In [42]:
famids = map(lambda fam : fam.id , animals.families[:10])

In [43]:
famnames = map(lambda fam : fam.name , animals.families[:10])

In [44]:
pres = map(lambda tree : tree.pseudoPresenceAbsence(famids,5),trees)

We will take some families. Doesn't matter which. For now the 10 most frequent.

In [45]:
famnames

[u'Muridae',
 u'Tyrannidae',
 u'Parulidae',
 u'Emberizidae',
 u'Turdidae',
 u'Icteridae',
 u'Troglodytidae',
 u'Picidae',
 u'Phrynosomatidae',
 u'Trochilidae']